<a href="https://colab.research.google.com/github/ahmadjan223/machine-learning/blob/main/w3d4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#import required libraries
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import glob
import os
import pandas as pd



In [4]:
#Unzip dataset
!unzip /content/Flowers.zip

Archive:  /content/Flowers.zip
   creating: Data/Sunflower/
  inflating: Data/Sunflower/Sunflower (1).jpg  
  inflating: Data/Sunflower/Sunflower (10).jpg  
  inflating: Data/Sunflower/Sunflower (100).jpg  
  inflating: Data/Sunflower/Sunflower (11).jpg  
  inflating: Data/Sunflower/Sunflower (12).jpg  
  inflating: Data/Sunflower/Sunflower (13).jpg  
  inflating: Data/Sunflower/Sunflower (14).jpg  
  inflating: Data/Sunflower/Sunflower (15).jpg  
  inflating: Data/Sunflower/Sunflower (16).jpg  
  inflating: Data/Sunflower/Sunflower (17).jpg  
  inflating: Data/Sunflower/Sunflower (18).jpg  
  inflating: Data/Sunflower/Sunflower (19).jpg  
  inflating: Data/Sunflower/Sunflower (2).jpg  
  inflating: Data/Sunflower/Sunflower (20).jpg  
  inflating: Data/Sunflower/Sunflower (21).jpg  
  inflating: Data/Sunflower/Sunflower (22).jpg  
  inflating: Data/Sunflower/Sunflower (23).jpg  
  inflating: Data/Sunflower/Sunflower (24).jpg  
  inflating: Data/Sunflower/Sunflower (25).jpg  
  inflatin

In [11]:

#Make data frames of data
data = '/content/Data'
categories = os.listdir(data)
print(categories)
X, y = [], []
for category in categories:
    path = os.path.join(data, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        X.append(img_path)
        y.append(category)

['Tulip', 'Sunflower']
['/content/Data/Tulip/Tulip (76).jpg', '/content/Data/Tulip/Tulip (100).jpg', '/content/Data/Tulip/Tulip (44).jpg', '/content/Data/Tulip/Tulip (19).jpg', '/content/Data/Tulip/Tulip (63).jpg', '/content/Data/Tulip/Tulip (58).jpg', '/content/Data/Tulip/Tulip (92).jpg', '/content/Data/Tulip/Tulip (53).jpg', '/content/Data/Tulip/Tulip (82).jpg', '/content/Data/Tulip/Tulip (24).jpg', '/content/Data/Tulip/Tulip (69).jpg', '/content/Data/Tulip/Tulip (64).jpg', '/content/Data/Tulip/Tulip (46).jpg', '/content/Data/Tulip/Tulip (30).jpg', '/content/Data/Tulip/Tulip (5).jpg', '/content/Data/Tulip/Tulip (85).jpg', '/content/Data/Tulip/Tulip (75).jpg', '/content/Data/Tulip/Tulip (27).jpg', '/content/Data/Tulip/Tulip (80).jpg', '/content/Data/Tulip/Tulip (13).jpg', '/content/Data/Tulip/Tulip (91).jpg', '/content/Data/Tulip/Tulip (87).jpg', '/content/Data/Tulip/Tulip (9).jpg', '/content/Data/Tulip/Tulip (96).jpg', '/content/Data/Tulip/Tulip (41).jpg', '/content/Data/Tulip/Tulip 

In [12]:
#Test train split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=101)


In [13]:
train_df = pd.DataFrame({'FileName':X_train, 'Category':y_train})
val_df = pd.DataFrame({'FileName':X_val, 'Category':y_val})


In [14]:
#Data Pre-processing with augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                  horizontal_flip=True,
                                  shear_range=0.2,
                                  zoom_range=0.2)

training_set = train_datagen.flow_from_dataframe(train_df,
                                                x_col='FileName',
                                                y_col='Category',
batch_size=32,
                                                class_mode='categorical',
                                                target_size=(224, 224))

validation_set = train_datagen.flow_from_dataframe(val_df,
                                                x_col='FileName',
                                                y_col='Category',
batch_size=32,
                                                class_mode='categorical',
                                                target_size=(224, 224))


Found 140 validated image filenames belonging to 2 classes.
Found 60 validated image filenames belonging to 2 classes.


In [23]:
# # Load the VGG16 model with pre-trained weights (excluding the top FC layers)
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

from tensorflow.keras.applications import ResNet50

# Load the ResNet50 model with pre-trained weights (excluding the top FC layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


94765736/94765736 [==============================] - 5s 0us/step


In [24]:
# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False


In [25]:
# Create a new model on top of the pre-trained model
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(units=len(categories), activation='softmax'))


In [26]:
# Compile the model
batch_size=32
model.compile(optimizer=Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [27]:
# Display model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_2 (Dense)             (None, 256)               25690368  
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 514       
                                                                 
Total params: 49278594 (187.98 MB)
Trainable params: 25690882 (98.00 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [28]:
# Train the model
history = model.fit(x=training_set, validation_data=validation_set, epochs=10)


Epoch 1/10
5/5 [==============================] - 13s 2s/step - loss: 10.8746 - accuracy: 0.5071 - val_loss: 15.0578 - val_accuracy: 0.4500
Epoch 2/10
5/5 [==============================] - 3s 576ms/step - loss: 8.8644 - accuracy: 0.5500 - val_loss: 7.2657 - val_accuracy: 0.5500
Epoch 3/10
5/5 [==============================] - 4s 849ms/step - loss: 6.1064 - accuracy: 0.5071 - val_loss: 1.1029 - val_accuracy: 0.6333
Epoch 4/10
5/5 [==============================] - 3s 585ms/step - loss: 5.7460 - accuracy: 0.5357 - val_loss: 3.5637 - val_accuracy: 0.5167
Epoch 5/10
5/5 [==============================] - 3s 628ms/step - loss: 5.3352 - accuracy: 0.6071 - val_loss: 1.0522 - val_accuracy: 0.6000
Epoch 6/10
5/5 [==============================] - 3s 595ms/step - loss: 3.1229 - accuracy: 0.5357 - val_loss: 0.8886 - val_accuracy: 0.6000
Epoch 7/10
5/5 [==============================] - 3s 528ms/step - loss: 1.6244 - accuracy: 0.5571 - val_loss: 0.6089 - val_accuracy: 0.7167
Epoch 8/10
5/5 [====

In [29]:
# Load the image you want to make predictions on
new_image_path = '/content/Data/Sunflower/Sunflower (16).jpg'
 # Replace with the path to your image "/content/Data/Tulip/Tulip (41).jpg"
img = image.load_img(new_image_path, target_size=(224, 224))
img_array = image.img_to_array(img)

# Expand dimensions to match batch size
img_array = np.expand_dims(img_array, axis=0)

# Preprocess the image (rescale pixel values)


In [30]:
img_array = img_array / 255.0

# Make predictions
predictions = model.predict(img_array)

# Get the predicted class label
predicted_class_index = np.argmax(predictions)
predicted_class = categories[predicted_class_index]
print("Predicted Class:", predicted_class)


1/1 [==============================] - 2s 2s/step
Predicted Class: Tulip
